#Kaggle Learn 用户房价竞赛

In [1]:
!unzip home-data-for-ml-course.zip && rm -rf home-data-for-ml-course.zip

Archive:  home-data-for-ml-course.zip
  inflating: data_description.txt    
  inflating: sample_submission.csv   
  inflating: sample_submission.csv.gz  
  inflating: test.csv                
  inflating: test.csv.gz             
  inflating: train.csv               
  inflating: train.csv.gz            


In [33]:
#数据基础清洗用于基准模型测试
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

def simple_clean(train_df, test_df):
    """
    Parameters:
    train_df: 训练数据DataFrame (包含SalePrice)
    test_df: 测试数据DataFrame (不包含SalePrice)

    Returns:
    clean_train_df: 清洗后的训练数据
    clean_test_df: 清洗后的测试数据
    """

    # 复制数据
    train_clean = train_df.copy()
    test_clean = test_df.copy()

    # 分离目标变量
    y_train = train_clean['SalePrice'].copy()
    X_train = train_clean.drop('SalePrice', axis=1)

    # 合并特征数据统一处理（不包含目标变量）
    all_features = pd.concat([X_train, test_clean], ignore_index=True)

    print(f"原始数据: 训练集{train_clean.shape}, 测试集{test_clean.shape}")
    print(f"缺失值统计: {all_features.isnull().sum().sum()}")

    # 1. 填充缺失值
    # 数值变量用中位数填充
    numeric_cols = all_features.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if all_features[col].isnull().sum() > 0:
            all_features[col] = all_features[col].fillna(all_features[col].median())

    # 分类变量用众数填充
    categorical_cols = all_features.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        if all_features[col].isnull().sum() > 0:
            all_features[col] = all_features[col].fillna(all_features[col].mode()[0])

    # 2. 编码分类变量
    le = LabelEncoder()
    for col in categorical_cols:
        all_features[col] = le.fit_transform(all_features[col].astype(str))

    # 3. 分离训练集和测试集特征
    train_len = len(X_train)
    X_train_clean = all_features[:train_len].copy()
    X_test_clean = all_features[train_len:].copy()

    # 重新组合训练集（加回目标变量）
    train_final = X_train_clean.copy()
    train_final['SalePrice'] = y_train.values

    print(f"清洗完成: 训练集{train_final.shape}, 测试集{X_test_clean.shape}")
    print(f"剩余缺失值: {train_final.isnull().sum().sum() + X_test_clean.isnull().sum().sum()}")

    return train_final, X_test_clean

def prepare_baseline_data():
    """
    准备基准模型数据的完整流程
    """
    # 读取数据
    data = pd.read_csv('train.csv').drop('Id', axis=1)
    test = pd.read_csv('test.csv').drop('Id', axis=1)
    test_id = pd.read_csv('test.csv')['Id']

    # 清洗数据
    train_clean, test_clean = simple_clean(data, test)

    # 分离特征和目标
    X = train_clean.drop('SalePrice', axis=1)
    y = train_clean['SalePrice']

    # 训练验证分割
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    print(f"\n最终数据:")
    print(f"X_train: {X_train.shape}")
    print(f"X_val: {X_val.shape}")
    print(f"X_test: {test_clean.shape}")

    return X_train, X_val, y_train, y_val, test_clean, test_id

# 使用示例
if __name__ == "__main__":
    # 直接替换您的代码
    X_train, X_val, y_train, y_val, X_test, test_id = prepare_baseline_data()


原始数据: 训练集(1460, 80), 测试集(1459, 79)
缺失值统计: 15707
清洗完成: 训练集(1460, 80), 测试集(1459, 79)
剩余缺失值: 0

最终数据:
X_train: (1168, 79)
X_val: (292, 79)
X_test: (1459, 79)


In [30]:
#设置一个随机森林基准模型评估特征提取
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

base_line = RandomForestRegressor(n_estimators=100, random_state=0)
baseline_score = cross_val_score(base_line, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
baseline_score = np.sqrt(-baseline_score.mean())
print(f"RMSE Baseline Score: {baseline_score:.0f}")

import numpy as np
from sklearn.metrics import make_scorer
def rmsle_score(y_true, y_pred):
    return np.sqrt(np.mean((np.log1p(y_pred) - np.log1p(y_true))**2))

# 基准的RMSLE（小数格式）
base_line = RandomForestRegressor(n_estimators=100, random_state=0)
rmsle_scorer = make_scorer(rmsle_score, greater_is_better=False)
baseline_score = cross_val_score(base_line, X_train, y_train, cv=5, scoring=rmsle_scorer)
baseline_score = -baseline_score.mean()

print(f"Baseline RMSLE: {baseline_score:.6f}")

RMSE Baseline Score: 30740
Baseline RMSLE: 0.146005


In [22]:
#特征重要性排序
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

def show_feature_importance(X_train, y_train, top_n=20):

    # 训练随机森林
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)

    # 获取并排序特征重要性
    importance_scores = pd.Series(
        rf.feature_importances_,
        index=X_train.columns
    ).sort_values(ascending=False)

    print(f"Top {top_n} 特征重要性:")
    for i, (feature, score) in enumerate(importance_scores.head(top_n).items(), 1):
        print(f"{i:2d}. {feature:<20} {score:.4f}")

    return importance_scores

importance_scores = show_feature_importance(X_train, y_train)

Top 20 特征重要性:
 1. OverallQual          0.5565
 2. GrLivArea            0.1214
 3. 2ndFlrSF             0.0354
 4. TotalBsmtSF          0.0346
 5. BsmtFinSF1           0.0288
 6. 1stFlrSF             0.0267
 7. LotArea              0.0177
 8. GarageArea           0.0161
 9. YearBuilt            0.0123
10. GarageCars           0.0118
11. Neighborhood         0.0091
12. LotFrontage          0.0077
13. GarageFinish         0.0072
14. BsmtQual             0.0070
15. TotRmsAbvGrd         0.0068
16. YearRemodAdd         0.0068
17. GarageYrBlt          0.0065
18. FullBath             0.0057
19. OpenPorchSF          0.0057
20. BsmtUnfSF            0.0052


In [36]:
#特征工程
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

def kaggle_winning_feature_engineering(df):
    df_new = df.copy()
    current_year = 2024

    # === 1. 语义化缺失值处理 ===
    none_features = {
        'PoolQC': 'NoPool', 'FireplaceQu': 'NoFireplace', 'GarageType': 'NoGarage',
        'GarageFinish': 'NoGarage', 'GarageQual': 'NoGarage', 'GarageCond': 'NoGarage',
        'BsmtQual': 'NoBasement', 'BsmtCond': 'NoBasement', 'BsmtExposure': 'NoBasement',
        'BsmtFinType1': 'NoBasement', 'BsmtFinType2': 'NoBasement', 'MiscFeature': 'None',
        'Alley': 'NoAlley', 'Fence': 'NoFence', 'MasVnrType': 'None'
    }

    for col, fill_value in none_features.items():
        if col in df_new.columns:
            df_new[col] = df_new[col].fillna(fill_value)

    # === 2. 序数特征正确编码 ===
    quality_map = {'NoPool': 0, 'NoFireplace': 0, 'NoBasement': 0, 'NoGarage': 0,
                   'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}

    quality_cols = ['PoolQC', 'FireplaceQu', 'BsmtQual', 'BsmtCond',
                   'GarageQual', 'GarageCond', 'ExterQual', 'ExterCond',
                   'HeatingQC', 'KitchenQual']

    for col in quality_cols:
        if col in df_new.columns:
            df_new[col] = df_new[col].map(quality_map).fillna(3)

    # === 3. 创建新特征 ===
    # 原有特征
    df_new['HouseAge'] = current_year - df_new['YearBuilt']
    df_new['QualityArea'] = df_new['OverallQual'] * (df_new['GrLivArea'] / 1000)
    df_new['TotalLivingArea'] = df_new['GrLivArea'] + df_new['TotalBsmtSF']
    df_new['QualityLotArea'] = df_new['OverallQual'] * (df_new['LotArea'] / 1000)
    df_new['AreaPerRoom'] = df_new['GrLivArea'] / (df_new['TotRmsAbvGrd'] + 1)
    df_new['GarageValue'] = (df_new['GarageCars'] * df_new['GarageArea'] / 100).fillna(0)
    df_new['FloorBalance'] = df_new['2ndFlrSF'] / (df_new['1stFlrSF'] + 1)
    df_new['BsmtFinishedRatio'] = df_new['BsmtFinSF1'] / (df_new['TotalBsmtSF'] + 1)

    # 获胜新特征
    df_new['HasFireplace'] = (df_new['Fireplaces'] > 0).astype(int)
    df_new['HasGarage'] = (df_new['GarageArea'] > 0).astype(int)
    df_new['HasBasement'] = (df_new['TotalBsmtSF'] > 0).astype(int)
    df_new['Has2ndFloor'] = (df_new['2ndFlrSF'] > 0).astype(int)

    df_new['TotalSF'] = df_new['TotalBsmtSF'] + df_new['1stFlrSF'] + df_new['2ndFlrSF']
    df_new['TotalBathrooms'] = (df_new['FullBath'] + df_new['HalfBath']*0.5 +
                               df_new['BsmtFullBath'] + df_new['BsmtHalfBath']*0.5)

    # 关键比例特征
    df_new['Living_Lot_Ratio'] = df_new['GrLivArea'] / (df_new['LotArea'] + 1)
    df_new['Basement_Ratio'] = df_new['TotalBsmtSF'] / (df_new['TotalSF'] + 1)
    df_new['Quality_TotalSF'] = df_new['OverallQual'] * df_new['TotalSF']

    # OverallCond特殊处理
    if 'OverallCond' in df_new.columns:
        df_new['Quality_Condition_Product'] = df_new['OverallQual'] * df_new['OverallCond']

    df_new['Is_Luxury'] = ((df_new['OverallQual'] >= 8) &
                          (df_new['GrLivArea'] > df_new['GrLivArea'].quantile(0.8))).astype(int)

    # === 4. 处理剩余变量 ===
    le = LabelEncoder()
    categorical_cols = df_new.select_dtypes(include=['object']).columns

    for col in categorical_cols:
        if df_new[col].isnull().sum() > 0:
            df_new[col] = df_new[col].fillna('Unknown')
        df_new[col] = le.fit_transform(df_new[col].astype(str))

    numeric_cols = df_new.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if df_new[col].isnull().sum() > 0:
            df_new[col] = df_new[col].fillna(0)

    # === 5. 特征选择 ===
    top_features = [
        'OverallQual', 'GrLivArea', '2ndFlrSF', 'TotalBsmtSF', 'BsmtFinSF1',
        '1stFlrSF', 'LotArea', 'GarageArea', 'YearBuilt', 'GarageCars',
        'Neighborhood', 'LotFrontage', 'GarageFinish', 'BsmtQual', 'TotRmsAbvGrd',
        'YearRemodAdd', 'GarageYrBlt', 'FullBath', 'OpenPorchSF', 'BsmtUnfSF'
    ]

    derived_features = [
        'HouseAge', 'QualityArea', 'TotalLivingArea', 'QualityLotArea',
        'AreaPerRoom', 'GarageValue', 'FloorBalance', 'BsmtFinishedRatio',
        'HasFireplace', 'HasGarage', 'HasBasement', 'Has2ndFloor',
        'TotalSF', 'TotalBathrooms', 'Living_Lot_Ratio', 'Basement_Ratio',
        'Quality_TotalSF', 'Quality_Condition_Product', 'Is_Luxury',
        'ExterQual', 'KitchenQual'
    ]

    all_features = top_features + derived_features
    available_features = [col for col in all_features if col in df_new.columns]

    return df_new[available_features]

def remove_outliers_simple(X_train, y_train):
    """简单异常值移除"""
    outlier_condition = (X_train['GrLivArea'] > 4000) & (y_train < 200000)
    outlier_indices = X_train[outlier_condition].index

    if len(outlier_indices) > 0:
        print(f"移除 {len(outlier_indices)} 个异常值")
        X_clean = X_train.drop(outlier_indices)
        y_clean = y_train.drop(outlier_indices)
        return X_clean, y_clean
    else:
        return X_train, y_train

# === 使用方法 ===
X_train_clean, y_train_clean = remove_outliers_simple(X_train, y_train)

fe6_train = kaggle_winning_feature_engineering(X_train_clean)
fe6_val = kaggle_winning_feature_engineering(X_val)
fe6_test = kaggle_winning_feature_engineering(X_test)

y_log = np.log1p(y_train_clean)
winning_model = RandomForestRegressor(n_estimators=200, random_state=0)
winning_scores = cross_val_score(winning_model, fe6_train, y_log, cv=5, scoring='neg_mean_squared_error')
winning_rmsle = np.sqrt(-winning_scores.mean())

print(f"获胜RMSLE: {winning_rmsle:.6f}")

移除 2 个异常值
获胜RMSLE: 0.137254


#尝试autogluon找到最适合的模型

In [60]:
from autogluon.tabular import TabularDataset, TabularPredictor

train_data = TabularDataset(fe6_train.join(y_train))
eval_data = TabularDataset(fe6_val.join(y_val))
test_data = TabularDataset(fe6_test)

In [61]:
from time import time
predictor = TabularPredictor(label='SalePrice')
predictor = predictor.fit(train_data=train_data,time_limit=1200,num_stack_levels=2,num_bag_folds=5)

No path specified. Models will be saved in: "AutogluonModels/ag-20250615_202309"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          8
Memory Avail:       47.85 GB / 50.99 GB (93.8%)
Disk Space Avail:   179.95 GB / 225.83 GB (79.7%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitio

In [62]:
predict = predictor.predict(eval_data)
predict.head()

,SalePrice
892,149008.093750
1105,315183.437500
413,110710.085938
522,161063.546875
1036,304477.875000


In [63]:
predictor.evaluate(eval_data)

{'root_mean_squared_error': np.float64(-39588.58936184447),
 'mean_squared_error': -1567256448.0,
 'mean_absolute_error': -17613.64453125,
 'r2': 0.7956727743148804,
 'pearsonr': 0.9083374909107125,
 'median_absolute_error': np.float64(-9084.5625)}

In [64]:
predictor.leaderboard(eval_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,NeuralNetFastAI_BAG_L2,-39232.606544,-16938.052013,root_mean_squared_error,0.617415,0.603595,43.547019,0.071779,0.044599,4.331790,2,True,18
1,NeuralNetFastAI_BAG_L1,-39257.339402,-17423.405677,root_mean_squared_error,0.067314,0.045205,6.461066,0.067314,0.045205,6.461066,1,True,8
2,NeuralNetFastAI_BAG_L3,-39271.598493,-17433.926774,root_mean_squared_error,1.111612,1.018285,84.229722,0.063217,0.042828,4.768364,3,True,28
3,WeightedEnsemble_L2,-39561.775774,-16559.598589,root_mean_squared_error,0.189175,0.189163,28.986216,0.002320,0.000452,0.012446,2,True,12
4,WeightedEnsemble_L4,-39588.589362,-16396.708773,root_mean_squared_error,0.682961,0.656796,61.708849,0.004350,0.000494,0.018235,4,True,32
5,NeuralNetTorch_BAG_L1,-39776.958308,-17012.818866,root_mean_squared_error,0.051453,0.054194,11.893754,0.051453,0.054194,11.893754,1,True,10
6,LightGBMXT_BAG_L1,-39880.549476,-17785.189610,root_mean_squared_error,0.051523,0.081041,4.009942,0.051523,0.081041,4.009942,1,True,3
7,WeightedEnsemble_L3,-40177.079631,-16560.916895,root_mean_squared_error,0.811050,0.774904,63.139332,0.002048,0.000395,0.012472,3,True,22
8,NeuralNetTorch_BAG_L2,-40833.718461,-17102.398525,root_mean_squared_error,0.606831,0.611703,57.358823,0.061195,0.052707,18.143594,2,True,20
9,NeuralNetTorch_BAG_L3,-40902.135938,-17319.975324,root_mean_squared_error,1.103291,1.027645,91.555552,0.054895,0.052188,12.094194,3,True,30


In [65]:
predict_test = predictor.predict(test_data)
predict_test.head()


,SalePrice
1460,123139.843750
1461,160048.984375
1462,185627.968750
1463,188616.328125
1464,185937.390625


In [66]:
test_id.head()

,Id
0,1461
1,1462
2,1463
3,1464
4,1465


In [67]:
#提交
submission = pd.DataFrame({'Id': test_id.values, 'SalePrice': predict_test.values})
submission.to_csv('submission.csv',index=False)